In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.models as models

In [141]:
class ResNet(nn.Module):
    def __init__(self, **params):
        super().__init__()
        
        self.dim = params['dim']
        self.expansion = params['expansion']
        self.num_classes = params['num_classes']
        
        # parameters from resnet paper
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=self.dim, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.dim)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # bottleneck layers
        self.layer1 = self._build_layer(num_layers=3, out_dim=64, stride=1, padding=1) 
        self.layer2 = self._build_layer(num_layers=4, out_dim=128, stride=2, padding=1)
        self.layer3 = self._build_layer(num_layers=6, out_dim=256, stride=2, padding=1)
        self.layer4 = self._build_layer(num_layers=3, out_dim=512, stride=2, padding=1)
        
        # pool/fc
        self.adapool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(2048, self.num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.adapool(x)
        x = x.flatten(1)
        out = self.fc(x)
        
        return out
         
    def _build_layer(self, num_layers, out_dim, stride, padding):
        '''
        layer1: (no image reduction)
            in_dim: 64
            out_dim: 64
            stride: 1
            padding: 0
        
        layer2: 56 -> 28
            in_dim: 256
            out_dim: 128
            stride: 2
            padding: 1
            
        layer3: 28 -> 14
            in_dim: 516
            out_dim: 256
            stride; 2
            padding: 1
        
        layer4: 14 -> 7
            in_dim: 1028
            out_dim: 512
            stride: 2
            padding: 1
            
        AdaptiveAvgPool2d: 7 -> 1
        '''
        # apply stride (image shape reduction) in first bottleneck block of each layer
        layers = []
        layers.append(
            Bottleneck(in_dim=self.dim, out_dim=out_dim, expansion=self.expansion, stride=stride, padding=padding, downsample=True)
        )
        
        self.dim = self.expansion * out_dim # dimension expansion
        for i in range(1, num_layers):
            layers.append(
                Bottleneck(in_dim=self.dim, out_dim=out_dim, expansion=self.expansion, stride=1, padding=padding)
            )
            
        return nn.Sequential(*layers)

In [142]:
# pytorch bottleneck architecture applies stride in conv3x3 with stride=2 and padding=1
# other architectures apply stride in 1st conv1x1 with stride=2 and padding=0

# apply stride (image shape reduction) in first bottleneck block of each layer

# bottleneck architecture: 
    # conv1x1 dimension reduction
    # conv3x3 image shape reduction
    # conv1x1 dimension expansion

class Bottleneck(nn.Module):
    def __init__(self, in_dim, out_dim, expansion, stride, padding, downsample=False):
        super().__init__()
        self.expansion = expansion
        
        # layers
        self.conv1 = nn.Conv2d(in_channels=in_dim, out_channels=out_dim, kernel_size=1, stride=1)
        self.bn1 = nn.BatchNorm2d(out_dim)
        self.conv2 = nn.Conv2d(in_channels=out_dim, out_channels=out_dim, kernel_size=3, stride=stride, padding=padding)
        self.bn2 = nn.BatchNorm2d(out_dim)
        self.conv3 = nn.Conv2d(in_channels=out_dim, out_channels=out_dim * self.expansion, kernel_size=1, stride=1)
        self.bn3 = nn.BatchNorm2d(out_dim * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        
        self.downsample = nn.Identity()
        if downsample:
            self.downsample = nn.Conv2d(in_channels=in_dim, out_channels=out_dim * self.expansion, kernel_size=1, stride=stride)
        
    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        
        identity = self.downsample(x) # identity mapping
        
        out += identity
        out = self.relu(out)
        
        return out

In [143]:
params = {'dim':64, 'expansion':4, 'num_classes':10}

In [144]:
model = ResNet(**params)

In [145]:
img = torch.rand(1, 3, 224, 224)

In [147]:
out = model(img)

In [152]:
out

tensor([[-0.6292, -0.0895, -0.2372,  0.1850, -0.9448, -0.8124, -0.6230, -0.8094,
          0.6364, -1.2386]], grad_fn=<AddmmBackward0>)